In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

vocab_size= 50257
output_dim = 256

token_emb_layer = torch.nn.Embedding(vocab_size, output_dim)

In [2]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

encode = tokenizer.encode("Hi everyone")
encode


[17250, 2506]

In [52]:
class GPTDatasetV1(Dataset):
    def __init__ (self,text, max_length, stride):
        self.input_ids =[]
        self.target_ids = []
        
        encodeText = tokenizer.encode(text)

        for i in range(0, len(encodeText) - max_length - 1, stride):
            chunk = encodeText[i: i + max_length + 1]
            input_chunks = chunk[:-1]
            target_chunks = chunk[1:]

            self.input_ids.append(torch.tensor(input_chunks))
            self.target_ids.append(torch.tensor(target_chunks))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

In [53]:
def createData(text, batch_size = 8, max_length = 4, stride = 4, shuffle=True):
    data = GPTDatasetV1(text, max_length, stride)
    
    preprocess = DataLoader(
        dataset=data,
        batch_size= batch_size,
        num_workers=4,
        shuffle = shuffle,
        drop_last=True
    )
    
    return preprocess


In [48]:
with open("the-verdict.txt") as f:
    text = f.read()
    
text[:300]


'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would ha'

In [54]:
data_iter = iter(createData(text))

inputs, target = next(data_iter)

print(inputs)
print(target)


tensor([[ 6000,  1517,   373,   284],
        [  373,   884,   281,  1167],
        [ 1110,    13,   198,   198],
        [ 1182,   287,   314,  2936],
        [  896,     6,   886,   314],
        [  921,  9670, 28022,    11],
        [  273,  1807,   673,   750],
        [ 5812,    11,  8759,  2763]])
tensor([[ 1517,   373,   284,   766],
        [  884,   281,  1167,  2588],
        [   13,   198,   198,    40],
        [  287,   314,  2936,   355],
        [    6,   886,   314,  5220],
        [ 9670, 28022,    11,   810],
        [ 1807,   673,   750,    13],
        [   11,  8759,  2763,  1043]])


In [55]:
tokens_emb = token_emb_layer(inputs)
print(tokens_emb.shape)

torch.Size([8, 4, 256])


In [64]:
context_length =4
pos_emb_layer = torch.nn.Embedding(context_length, output_dim)

pos_emb = pos_emb_layer(torch.arange(4))
# print(pos_emb)
print(torch.arange(4))

tensor([0, 1, 2, 3])


In [58]:
input_emb = tokens_emb + pos_emb
print(input_emb.shape)

torch.Size([8, 4, 256])
